In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils import shuffle
from PIL import Image
from sklearn.preprocessing import LabelBinarizer

# Function to filter out non-image files
def is_image_file(filename):
    """
    Checks if a filename represents an image file.

    Args:
    - filename (str): The name of the file to check.

    Returns:
    - bool: True if the filename represents an image file, False otherwise.
    """
    return any(filename.endswith(extension) for extension in ['.png', '.jpg', '.jpeg'])

# Function to load and preprocess images
def load_and_preprocess_image(image_path, target_size):
    """
    Loads and preprocesses an image from a given path.

    Args:
    - image_path (str): The path to the image file.
    - target_size (tuple): The target size for the loaded image.

    Returns:
    - numpy.ndarray: The preprocessed image as a NumPy array.
    """
    img = load_img(image_path, target_size=target_size)
    img_array = img_to_array(img)
    return img_array / 255.0

# Define paths and parameters
data_path_train = "cars_train"  # path to train dataset
data_path_test = "cars_test"  # path to test dataset
image_size = (128, 128)
batch_size = 32
epochs = 10

# Data preprocessing and cleaning for training set
image_list_train = []
class_labels_train = []

for class_label in os.listdir(data_path_train):
    class_path = os.path.join(data_path_train, class_label)
    if os.path.isdir(class_path):
        for filename in os.listdir(class_path):
            if is_image_file(filename):
                image_path = os.path.join(class_path, filename)
                image_list_train.append(image_path)
                class_labels_train.append(class_label)

# Data preprocessing and cleaning for test set
image_list_test = []
class_labels_test = []

for class_label in os.listdir(data_path_test):
    class_path = os.path.join(data_path_test, class_label)
    if os.path.isdir(class_path):
        for filename in os.listdir(class_path):
            if is_image_file(filename):
                image_path = os.path.join(class_path, filename)
                image_list_test.append(image_path)
                class_labels_test.append(class_label)

# Shuffle the data
image_list_train, class_labels_train = shuffle(image_list_train, class_labels_train, random_state=42)
image_list_test, class_labels_test = shuffle(image_list_test, class_labels_test, random_state=42)

# Split the dataset into training and validation sets
train_images, validation_images, train_labels, validation_labels = train_test_split(
    image_list_train, class_labels_train, test_size=0.2, random_state=42, stratify=class_labels_train
)

# Convert images to numpy arrays for training and validation sets
X_train = np.array([load_and_preprocess_image(img, image_size) for img in train_images])
X_validation = np.array([load_and_preprocess_image(img, image_size) for img in validation_images])
X_test = np.array([load_and_preprocess_image(img, image_size) for img in image_list_test])

# One-hot encode the labels for training and validation sets
label_binarizer = LabelBinarizer()
y_train = label_binarizer.fit_transform(train_labels)
y_validation = label_binarizer.transform(validation_labels)
y_test = label_binarizer.transform(class_labels_test)

# Model architecture
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(image_size[0], image_size[1], 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(len(label_binarizer.classes_), activation='softmax'))

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=epochs, validation_data=(X_validation, y_validation))

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"\nTest Accuracy: {test_accuracy*100:.2f}%")

# Print classification report and confusion matrix for the test set
test_predictions = model.predict(X_test)
test_predicted_classes = np.argmax(test_predictions, axis=1)

true_classes_test = label_binarizer.transform(class_labels_test)
class_labels_test = list(label_binarizer.classes_)

print("\nClassification Report (Test Set):\n")
print(classification_report(np.argmax(true_classes_test, axis=1), test_predicted_classes, target_names=class_labels_test))

print("\nConfusion Matrix (Test Set):\n")
print(confusion_matrix(np.argmax(true_classes_test, axis=1), test_predicted_classes))

# Plot training history
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()


204/204 [==============================] - 173s 835ms/step - loss: 0.0033 - accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00

Test Accuracy: 0.00%
252/252 [==============================] - 52s 203ms/step

Classification Report (Test Set):

              precision    recall  f1-score   support

  cars_train       1.00      1.00      1.00      8041

    accuracy                           1.00      8041
   macro avg       1.00      1.00      1.00      8041
weighted avg       1.00      1.00      1.00      8041


Confusion Matrix (Test Set):

[[8041]]
